In [2]:
## Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
## Importing the OS and JSON Modules
import os,json

In [3]:
SI = pd.read_csv('superhero_info - superhero_info.csv')
SI.head()

,Hero|Publisher,Gender,Race,Alignment,Hair color,Eye color,Skin color,Measurements
0,A-Bomb|Marvel Comics,Male,Human,good,No Hair,yellow,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}"
1,Abe Sapien|Dark Horse Comics,Male,Icthyo Sapien,good,No Hair,blue,blue,"{'Height': '191.0 cm', 'Weight': '65.0 kg'}"
2,Abin Sur|DC Comics,Male,Ungaran,good,No Hair,blue,red,"{'Height': '185.0 cm', 'Weight': '90.0 kg'}"
3,Abomination|Marvel Comics,Male,Human / Radiation,bad,No Hair,green,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}"
4,Absorbing Man|Marvel Comics,Male,Human,bad,No Hair,blue,Unknown,"{'Height': '193.0 cm', 'Weight': '122.0 kg'}"


In [4]:
SP = pd.read_csv('superhero_powers - superhero_powers.csv')
SP.head()

,hero_names,Powers
0,3-D Man,"Agility,Super Strength,Stamina,Super Speed"
1,A-Bomb,"Accelerated Healing,Durability,Longevity,Super..."
2,Abe Sapien,"Agility,Accelerated Healing,Cold Resistance,Du..."
3,Abin Sur,Lantern Power Ring
4,Abomination,"Accelerated Healing,Intelligence,Super Strengt..."


I. Clean the files and combine them into one final DataFrame.

In [5]:
# Exploring existing format for Hero|Publisher with a few examples
SI['Hero|Publisher'].head(2)

0            A-Bomb|Marvel Comics
1    Abe Sapien|Dark Horse Comics
Name: Hero|Publisher, dtype: object

In [6]:
## To split, add expand=True
SI['Hero|Publisher'].str.split('|',expand=True)

,0,1
0,A-Bomb,Marvel Comics
1,Abe Sapien,Dark Horse Comics
2,Abin Sur,DC Comics
3,Abomination,Marvel Comics
4,Absorbing Man,Marvel Comics
...,...,...
458,Yellowjacket,Marvel Comics
459,Yellowjacket II,Marvel Comics
460,Yoda,George Lucas
461,Zatanna,DC Comics


In [7]:
## save the 2 new columns into the dataframe
SI[['Hero','Publisher']] = SI['Hero|Publisher'].str.split('|',expand=True)
SI.head(2)

,Hero|Publisher,Gender,Race,Alignment,Hair color,Eye color,Skin color,Measurements,Hero,Publisher
0,A-Bomb|Marvel Comics,Male,Human,good,No Hair,yellow,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}",A-Bomb,Marvel Comics
1,Abe Sapien|Dark Horse Comics,Male,Icthyo Sapien,good,No Hair,blue,blue,"{'Height': '191.0 cm', 'Weight': '65.0 kg'}",Abe Sapien,Dark Horse Comics


In [8]:
## drop the original column 
SI = SI.drop(columns=['Hero|Publisher'])
SI.head(2)

,Gender,Race,Alignment,Hair color,Eye color,Skin color,Measurements,Hero,Publisher
0,Male,Human,good,No Hair,yellow,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}",A-Bomb,Marvel Comics
1,Male,Icthyo Sapien,good,No Hair,blue,blue,"{'Height': '191.0 cm', 'Weight': '65.0 kg'}",Abe Sapien,Dark Horse Comics


In [9]:
## examining a single value from the measurements col
mesmt = SI.loc[0,"Measurements"]
print(type(mesmt))
mesmt

<class 'str'>


"{'Height': '203.0 cm', 'Weight': '441.0 kg'}"

In [10]:
import json
json.loads(mesmt)

JSONDecodeError: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)

In [11]:
#To fix this, we can replace our single quotes with double quotes before we use json.loads
mesmt = mesmt.replace("'",'"')
mesmt

'{"Height": "203.0 cm", "Weight": "441.0 kg"}'

In [12]:
## now we can use json.loads
fixed_mesmt = json.loads(mesmt)
print(type(fixed_mesmt))
fixed_mesmt

<class 'dict'>


{'Height': '203.0 cm', 'Weight': '441.0 kg'}

In [13]:
fixed_mesmt_2 = fixed_mesmt.replace("'",'')
fixed_mesmt_2

AttributeError: 'dict' object has no attribute 'replace'

In [14]:
#We can apply this transformation to the entire column by combining:
#Using .str.replace to replace single quotes.
#Using .apply with json.loads to convert all rows simultaneously   

## use .str.replace to replace all single quotes
SI['Measurements'] = SI['Measurements'].str.replace("'",'"')
## Apply the json.loads to the full column
SI['Measurements'] = SI['Measurements'].apply(json.loads)
SI['Measurements'].head()

0    {'Height': '203.0 cm', 'Weight': '441.0 kg'}
1     {'Height': '191.0 cm', 'Weight': '65.0 kg'}
2     {'Height': '185.0 cm', 'Weight': '90.0 kg'}
3    {'Height': '203.0 cm', 'Weight': '441.0 kg'}
4    {'Height': '193.0 cm', 'Weight': '122.0 kg'}
Name: Measurements, dtype: object

In [15]:
## check a single value after transformation
test_mesmt = SI.loc[0, 'Measurements']
print(type(test_mesmt))
test_mesmt

<class 'dict'>


{'Height': '203.0 cm', 'Weight': '441.0 kg'}

In [16]:
#Unpack a column of Measurements into separate columns
hei_wei = SI['Measurements'].apply(pd.Series)
hei_wei

,Height,Weight
0,203.0 cm,441.0 kg
1,191.0 cm,65.0 kg
2,185.0 cm,90.0 kg
3,203.0 cm,441.0 kg
4,193.0 cm,122.0 kg
...,...,...
458,183.0 cm,83.0 kg
459,165.0 cm,52.0 kg
460,66.0 cm,17.0 kg
461,170.0 cm,57.0 kg


In [17]:
# concat hei_wei with original dataframe
SI = pd.concat((SI, hei_wei), axis = 1)
SI.head(2)

,Gender,Race,Alignment,Hair color,Eye color,Skin color,Measurements,Hero,Publisher,Height,Weight
0,Male,Human,good,No Hair,yellow,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}",A-Bomb,Marvel Comics,203.0 cm,441.0 kg
1,Male,Icthyo Sapien,good,No Hair,blue,blue,"{'Height': '191.0 cm', 'Weight': '65.0 kg'}",Abe Sapien,Dark Horse Comics,191.0 cm,65.0 kg


In [18]:
#drop the original Measurements column.
SI = SI.drop(columns=['Measurements'])

In [19]:
SI.head()

,Gender,Race,Alignment,Hair color,Eye color,Skin color,Hero,Publisher,Height,Weight
0,Male,Human,good,No Hair,yellow,Unknown,A-Bomb,Marvel Comics,203.0 cm,441.0 kg
1,Male,Icthyo Sapien,good,No Hair,blue,blue,Abe Sapien,Dark Horse Comics,191.0 cm,65.0 kg
2,Male,Ungaran,good,No Hair,blue,red,Abin Sur,DC Comics,185.0 cm,90.0 kg
3,Male,Human / Radiation,bad,No Hair,green,Unknown,Abomination,Marvel Comics,203.0 cm,441.0 kg
4,Male,Human,bad,No Hair,blue,Unknown,Absorbing Man,Marvel Comics,193.0 cm,122.0 kg


In [20]:
#one-hot-encoded columns for every power that appears in the dataset.
SP['Powers'].value_counts()

Intelligence                                                                                                                                                                                                                                                         8
Durability,Super Strength                                                                                                                                                                                                                                            5
Agility,Stealth,Marksmanship,Weapons Master,Stamina                                                                                                                                                                                                                  4
Marksmanship                                                                                                                                                                                                       

In [21]:
## showing the lists are really strings
SP.loc[2,'Powers']

'Agility,Accelerated Healing,Cold Resistance,Durability,Underwater breathing,Marksmanship,Weapons Master,Longevity,Intelligence,Super Strength,Telepathy,Stamina,Immortality,Reflexes,Enhanced Sight,Sub-Mariner'

In [22]:
SP.head()

,hero_names,Powers
0,3-D Man,"Agility,Super Strength,Stamina,Super Speed"
1,A-Bomb,"Accelerated Healing,Durability,Longevity,Super..."
2,Abe Sapien,"Agility,Accelerated Healing,Cold Resistance,Du..."
3,Abin Sur,Lantern Power Ring
4,Abomination,"Accelerated Healing,Intelligence,Super Strengt..."


In [23]:
## To split, add expand=True
SP['PowerSplit'] = SP['Powers'].str.split(',',expand=False)

In [24]:
## exploding the column of lists
exploded = SP.explode('PowerSplit')
exploded

,hero_names,Powers,PowerSplit
0,3-D Man,"Agility,Super Strength,Stamina,Super Speed",Agility
0,3-D Man,"Agility,Super Strength,Stamina,Super Speed",Super Strength
0,3-D Man,"Agility,Super Strength,Stamina,Super Speed",Stamina
0,3-D Man,"Agility,Super Strength,Stamina,Super Speed",Super Speed
1,A-Bomb,"Accelerated Healing,Durability,Longevity,Super...",Accelerated Healing
...,...,...,...
665,Zatanna,"Cryokinesis,Telepathy,Magic,Fire Control,Proba...",Weather Control
666,Zoom,"Super Speed,Intangibility,Time Travel,Time Man...",Super Speed
666,Zoom,"Super Speed,Intangibility,Time Travel,Time Man...",Intangibility
666,Zoom,"Super Speed,Intangibility,Time Travel,Time Man...",Time Travel


In [25]:
## saving the unique values from the exploded column
cols_to_make = exploded['PowerSplit'].dropna().unique()
cols_to_make

array(['Agility', 'Super Strength', 'Stamina', 'Super Speed',
       'Accelerated Healing', 'Durability', 'Longevity', 'Camouflage',
       'Self-Sustenance', 'Cold Resistance', 'Underwater breathing',
       'Marksmanship', 'Weapons Master', 'Intelligence', 'Telepathy',
       'Immortality', 'Reflexes', 'Enhanced Sight', 'Sub-Mariner',
       'Lantern Power Ring', 'Invulnerability', 'Animation',
       'Super Breath', 'Dimensional Awareness', 'Flight', 'Size Changing',
       'Teleportation', 'Magic', 'Dimensional Travel',
       'Molecular Manipulation', 'Energy Manipulation', 'Power Cosmic',
       'Energy Absorption', 'Elemental Transmogrification',
       'Fire Resistance', 'Natural Armor', 'Heat Resistance',
       'Matter Absorption', 'Regeneration', 'Stealth', 'Power Suit',
       'Energy Blasts', 'Energy Beams', 'Heat Generation', 'Danger Sense',
       'Phasing', 'Force Fields', 'Hypnokinesis', 'Invisibility',
       'Enhanced Senses', 'Jump', 'Shapeshifting', 'Elasticity',
 

In [33]:
for col in cols_to_make:
    SP[col] = SP['Powers'].str.contains(col)
SP.head()


,hero_names,Powers,PowerSplit,Agility,Super Strength,Stamina,Super Speed,Accelerated Healing,Durability,Longevity,...,Weather Control,Omnipresent,Omniscient,Hair Manipulation,Nova Force,Odin Force,Phoenix Force,Intuitive aptitude,Melting,Changing Armor
0,3-D Man,"Agility,Super Strength,Stamina,Super Speed","[Agility, Super Strength, Stamina, Super Speed]",True,True,True,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,A-Bomb,"Accelerated Healing,Durability,Longevity,Super...","[Accelerated Healing, Durability, Longevity, S...",False,True,True,False,True,True,True,...,False,False,False,False,False,False,False,False,False,False
2,Abe Sapien,"Agility,Accelerated Healing,Cold Resistance,Du...","[Agility, Accelerated Healing, Cold Resistance...",True,True,True,False,True,True,True,...,False,False,False,False,False,False,False,False,False,False
3,Abin Sur,Lantern Power Ring,[Lantern Power Ring],False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,Abomination,"Accelerated Healing,Intelligence,Super Strengt...","[Accelerated Healing, Intelligence, Super Stre...",False,True,True,True,True,False,False,...,False,False,False,False,False,False,False,False,False,False


In [34]:
## merge with our 2 dataframes
merged_df=pd.merge(SI, SP, left_on='Hero', right_on='hero_names', how='inner')
merged_df.head()

,Gender,Race,Alignment,Hair color,Eye color,Skin color,Hero,Publisher,Height,Weight,...,Weather Control,Omnipresent,Omniscient,Hair Manipulation,Nova Force,Odin Force,Phoenix Force,Intuitive aptitude,Melting,Changing Armor
0,Male,Human,good,No Hair,yellow,Unknown,A-Bomb,Marvel Comics,203.0 cm,441.0 kg,...,False,False,False,False,False,False,False,False,False,False
1,Male,Icthyo Sapien,good,No Hair,blue,blue,Abe Sapien,Dark Horse Comics,191.0 cm,65.0 kg,...,False,False,False,False,False,False,False,False,False,False
2,Male,Ungaran,good,No Hair,blue,red,Abin Sur,DC Comics,185.0 cm,90.0 kg,...,False,False,False,False,False,False,False,False,False,False
3,Male,Human / Radiation,bad,No Hair,green,Unknown,Abomination,Marvel Comics,203.0 cm,441.0 kg,...,False,False,False,False,False,False,False,False,False,False
4,Male,Human,bad,No Hair,blue,Unknown,Absorbing Man,Marvel Comics,193.0 cm,122.0 kg,...,False,False,False,False,False,False,False,False,False,False


In [39]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 463 entries, 0 to 462
Columns: 180 entries, Gender to Changing Armor
dtypes: bool(167), object(13)
memory usage: 126.1+ KB


In [40]:
#What is the average height of heroes for each publisher?

merged_df.groupby('Publisher')['Height'].mean()


TypeError: Could not convert 185.0 cm185.0 cm180.0 cm257.0 cm183.0 cm178.0 cm185.0 cm198.0 cm178.0 cm168.0 cm183.0 cm203.0 cm170.0 cm165.0 cm168.0 cm188.0 cm178.0 cm178.0 cm173.0 cm188.0 cm191.0 cm191.0 cm165.0 cm170.0 cm185.0 cm188.0 cm183.0 cm196.0 cm198.0 cm170.0 cm170.0 cm193.0 cm193.0 cm175.0 cm175.0 cm163.0 cm170.0 cm175.0 cm183.0 cm198.0 cm267.0 cm183.0 cm185.0 cm193.0 cm188.0 cm175.0 cm244.0 cm185.0 cm168.0 cm193.0 cm188.0 cm180.0 cm183.0 cm183.0 cm157.0 cm62.5 cm198.0 cm178.0 cm188.0 cm188.0 cm188.0 cm170.0 cm185.0 cm175.0 cm185.0 cm180.0 cm180.0 cm170.0 cm185.0 cm196.0 cm173.0 cm244.0 cm234.0 cm64.0 cm180.0 cm188.0 cm165.0 cm155.0 cm191.0 cm229.0 cm201.0 cm180.0 cm175.0 cm196.0 cm185.0 cm185.0 cm183.0 cm178.0 cm183.0 cm193.0 cm178.0 cm178.0 cm157.0 cm168.0 cm185.0 cm168.0 cm168.0 cm180.0 cm180.0 cm188.0 cm193.0 cm165.0 cm178.0 cm180.0 cm183.0 cm165.0 cm185.0 cm185.0 cm178.0 cm183.0 cm165.0 cm137.0 cm168.0 cm183.0 cm173.0 cm201.0 cm175.0 cm279.0 cm188.0 cm193.0 cm165.0 cm170.0

In [41]:
#Compare the average weight of super powers who have Super Speed to those who do not.
SuperSpeed_Y=merged_df['Super Speed'] == 'True'
SuperSpeed_N=merged_df['Super Speed'] == 'False'



In [42]:
SuperSpeed_Y

0      False
1      False
2      False
3      False
4      False
       ...  
458    False
459    False
460    False
461    False
462    False
Name: Super Speed, Length: 463, dtype: bool

In [43]:
merged_df.loc[SuperSpeed_Y]

,Gender,Race,Alignment,Hair color,Eye color,Skin color,Hero,Publisher,Height,Weight,...,Weather Control,Omnipresent,Omniscient,Hair Manipulation,Nova Force,Odin Force,Phoenix Force,Intuitive aptitude,Melting,Changing Armor


In [44]:
SuperSpeed_N

0      False
1      False
2      False
3      False
4      False
       ...  
458    False
459    False
460    False
461    False
462    False
Name: Super Speed, Length: 463, dtype: bool

In [45]:
merged_df.loc[SuperSpeed_N]

,Gender,Race,Alignment,Hair color,Eye color,Skin color,Hero,Publisher,Height,Weight,...,Weather Control,Omnipresent,Omniscient,Hair Manipulation,Nova Force,Odin Force,Phoenix Force,Intuitive aptitude,Melting,Changing Armor


In [46]:
merged_df['Super Speed'].value_counts()

False    265
True     198
Name: Super Speed, dtype: int64